In [1]:
!pip install transformers datasets torch

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
from datasets import load_dataset

dataset = load_dataset("dair-ai/emotion")

dataset


/media/nas_mount/avinash_ocr/.conda/envs_dirs/llma1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/media/nas_mount/avinash_ocr/.conda/envs_dirs/llma1/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for dair-ai/emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/dair-ai/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Generating test split: 100%|██████████| 2000/2000 [00:00<00:00, 46575.43 examples/s]


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")

def tokenize_function(examples):
    # return tokenizer(examples["text"], truncation=True)
    return tokenizer(examples["text"], padding='max_length', truncation=True, max_length=21)

tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map: 100%|██████████| 2000/2000 [00:00<00:00, 30201.57 examples/s]


In [8]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-chat-hf")


Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.92s/it]


In [22]:
from transformers import Trainer, TrainingArguments, deepspeed

deepspeed_config = {
    "zero_optimization": {
        "stage": 2
    }
}

In [24]:
from transformers import TrainingArguments

# training_args = TrainingArguments(
#     output_dir="./results",
#     evaluation_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     logging_dir='./logs',
#     logging_steps=10,
# )


training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,  # reduce batch size
    per_device_eval_batch_size=2,  # reduce batch size
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    
    gradient_accumulation_steps=4, 
    fp16=True,  # Enable mixed precision
)


In [ ]:
# def tokenize_function(examples):
#     return tokenizer(examples["text"], padding='max_length', truncation=True, max_length=21)

# tokenized_datasets = dataset.map(tokenize_function, batched=True)


In [17]:
import torch
torch.cuda.empty_cache()

import gc
gc.collect()

4061

In [19]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 5            |        cudaMalloc retries: 5         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |  48321 MiB |  48321 MiB |  48321 MiB |    512 B   |\n|       from large pool |  48320 MiB |  48320 MiB |  48320 MiB |      0 B   |\n|       from small pool |      1 MiB |      1 MiB |      1 MiB |    512 B   |\n|---------------------------------------------------------------------------|\n| Active memory         |  48321 MiB |  48321 MiB |  48321 MiB |    512 B   |\n|       from large pool |  48320 MiB |  48320 MiB |

In [25]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
)

trainer.train()


OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB. GPU 0 has a total capacity of 47.54 GiB of which 72.75 MiB is free. Including non-PyTorch memory, this process has 47.46 GiB memory in use. Of the allocated memory 47.19 GiB is allocated by PyTorch, and 16.08 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
results = trainer.evaluate()
print(results)
